Lựa chọn thuật toán phù hợp trong “Machine Learning
with Python”   
(chọn ít nhất 3 thuật toán để thực hiện và
đánh giá )

In [92]:
#import thư viện
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [93]:
#đọc dữ liệu
df1 =pd.read_csv('Danh_gia_cleaned.csv')

In [94]:
# kiểm tra dữ liệu bị thiếu
print(df1.isnull().sum())

ma_khach_hang            0
noi_dung_binh_luan       0
ngay_binh_luan           0
gio_binh_luan            0
so_sao                   0
ma_san_pham              0
content_new           1691
dtype: int64


In [95]:
# thay thế giá trị thiếu bằng chuỗi rỗng
df1['content_new']=df1['content_new'].fillna('')


In [96]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20674 entries, 0 to 20673
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ma_khach_hang       20674 non-null  int64 
 1   noi_dung_binh_luan  20674 non-null  object
 2   ngay_binh_luan      20674 non-null  object
 3   gio_binh_luan       20674 non-null  object
 4   so_sao              20674 non-null  int64 
 5   ma_san_pham         20674 non-null  int64 
 6   content_new         20674 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.1+ MB


In [97]:
#xem vài dòng
df1.head(10)

ma_khach_hang                                 noi_dung_binh_luan  \
0            443   SỬ DỤNG DỄ DÀNG, RẤT THOẢI MÁI, THƯ GIÃN TỘT ĐỘ.   
1           1030      Sử dụng dễ dãng,rất thoải mái,thư giãn tột độ   
2            689          Mình rất thích hasaki va sp tẩy trang này   
3           2519  Sản phẩm có khả năng làm sạch tốt. Lớp trang đ...   
4            402       Sữa rửa mặt tốt,sạch mụn,mịn da,đáng mua nha   
5           2517  Sau 77 49 dòng srm, thì chân ái là đây, mua tu...   
6           1750  Đó giờ mình sài bha của obagi cũng 3-4 chai rồ...   
7           3394                         Rất ok mình xài 2 chai rồi   
8           1817  Mik bị kich ứng, không hợp sản phẩm, sd được 3...   
9           3556  nhân viên tư vấn như mới train và được nhận vì...   

  ngay_binh_luan gio_binh_luan  so_sao  ma_san_pham  \
0     29/04/2023         17:06       5    308500015   
1     30/04/2023         15:04       5    308500015   
2     30/04/2023         18:34       5    422216594   
3     17/07/2022         13:48       5    204100075   
4     15/04/2023         23:04       5    422208977   
5     25/12/2022         17:11       5    204100019   
6       9/7/2022          4:34       5    100220035   
7       7/7/2022         11:18       5    204100024   
8       5/2/2024          3:50       4    422216354   
9      12/9/2023         22:33       5    100250032   

                                         content_new  
0                                  dễ_dàng thoải_mái  
1                                     dãng thoải_mái  
2                                   thích hasaki_tẩy  
3  sạch tốt loại_bỏ bẩn dầu_thừa kích_ứng sạch mà...  
4                                   rửa tốt sạch mịn  
5                            chân_ái thử khô khô nổi  
6                               sài đổi kèm tặng trừ  
7                                         đồng_ý xài  
8                             ứng không_hợp sản_phẩm  
9            tư_vấn đẹp thôi_á dán trả_lời không_như

In [98]:
# làm thống kê
df1.so_sao.describe()

count    20674.000000
mean         4.623634
std          0.889168
min          1.000000
25%          5.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: so_sao, dtype: float64

Nhận xét: số sao từ 1 đến 5 và điểm trung bình là 4.623634, độ lệch chuẩn nhỏ

In [99]:
df1.so_sao.value_counts()
# điểm 1,2,3 rất là ít. các điểm 4,5 người ta khen nhiều 

so_sao
5    16506
4     2144
3      979
1      544
2      501
Name: count, dtype: int64

In [100]:
#ta xem:1,2,3: không hài lòng và 4,5 : là hài lòng
# nếu số sao <=3 thì bằng 0 và ngược lại
df1['sentiment'] = np.where(df1['so_sao']<=3, 1, 0)


In [101]:
# xem lại dữ liệu vài dòng đầu tiên 
df1.head(20)

ma_khach_hang                                 noi_dung_binh_luan  \
0             443   SỬ DỤNG DỄ DÀNG, RẤT THOẢI MÁI, THƯ GIÃN TỘT ĐỘ.   
1            1030      Sử dụng dễ dãng,rất thoải mái,thư giãn tột độ   
2             689          Mình rất thích hasaki va sp tẩy trang này   
3            2519  Sản phẩm có khả năng làm sạch tốt. Lớp trang đ...   
4             402       Sữa rửa mặt tốt,sạch mụn,mịn da,đáng mua nha   
5            2517  Sau 77 49 dòng srm, thì chân ái là đây, mua tu...   
6            1750  Đó giờ mình sài bha của obagi cũng 3-4 chai rồ...   
7            3394                         Rất ok mình xài 2 chai rồi   
8            1817  Mik bị kich ứng, không hợp sản phẩm, sd được 3...   
9            3556  nhân viên tư vấn như mới train và được nhận vì...   
10           2375      Mùi thơm dịu nhe , mát nhanh, thư giãn tột dộ   
11           2027  Mình khum đánh giá cao cái vụ nâng tông trắng ...   
12           2928  dù là hay được so sánh với BNBG nhma mình thấy...   
13           3059  Miếng bông quá mỏng. thấm có chút nước tẩy tra...   
14           1118  Xài tốt, tẩy trang sạch, mình xài thấy thích h...   
15           2273                               Hơi bết và trắng quá   
16            736                                       sản phẩm tốt   
17            731  Mua chai thứ 4 5 gì rồi, đúng kiểu tiền nào củ...   
18           4234  rít mặt nên đắp xomg phải lau lại rôid dùng dư...   
19           2696                                        quá ok luôn   

   ngay_binh_luan gio_binh_luan  so_sao  ma_san_pham  \
0      29/04/2023         17:06       5    308500015   
1      30/04/2023         15:04       5    308500015   
2      30/04/2023         18:34       5    422216594   
3      17/07/2022         13:48       5    204100075   
4      15/04/2023         23:04       5    422208977   
5      25/12/2022         17:11       5    204100019   
6        9/7/2022          4:34       5    100220035   
7        7/7/2022         11:18       5    204100024   
8        5/2/2024          3:50       4    422216354   
9       12/9/2023         22:33       5    100250032   
10     29/04/2023         16:52       5    422205163   
11     24/10/2022         18:02       5    249500019   
12     27/03/2024         22:14       5    422209718   
13     19/12/2020          7:52       2    413400001   
14     22/06/2021          9:30       5    422204884   
15     17/12/2022         11:20       4    422208183   
16      5/12/2022         15:52       5    100230056   
17       4/3/2023         19:08       5    100230056   
18     17/07/2023         23:05       4    422205637   
19     18/08/2019         23:21       5    422203776   

                                          content_new  sentiment  
0                                   dễ_dàng thoải_mái          0  
1                                      dãng thoải_mái          0  
2                                    thích hasaki_tẩy          0  
3   sạch tốt loại_bỏ bẩn dầu_thừa kích_ứng sạch mà...          0  
4                                    rửa tốt sạch mịn          0  
5                             chân_ái thử khô khô nổi          0  
6                                sài đổi kèm tặng trừ          0  
7                                          đồng_ý xài          0  
8                              ứng không_hợp sản_phẩm          0  
9             tư_vấn đẹp thôi_á dán trả_lời không_như          0  
10                                                mát          0  
11  khum nâng trắng lắm chỉnh chống làm_lót phê_hợ...          0  
12      khô sánh nhma lừa tốt_trọng tầm_giá thoải_mái          0  
13            mỏng thấm tẩy nhũn liền tưa đáp_ứng tốt          1  
14  xài sạch xài thích không_bị châm chích xài trô...          0  
15                                      hơi bết trắng          0  
16                                                tốt          0  
17                                     xài đồng_ý hơi          0  
18                               rít đắp 

In [102]:
df1[df1['so_sao']==3]

ma_khach_hang                                 noi_dung_binh_luan  \
36              2723  Dạng sữa thắm hơi lâu và rất rất rất là cay lu...   
53              2528  sản phẩm cho da dầu thôi, dầu mụn nên cân nhắc...   
69              1370  cấp ẩm ổn, những vết thâm mới thì làm mờ tốt, ...   
86              1394  Mình da dầu mụn, mới mua xài thấy mát nhẹ mặt,...   
104             1453  Mình da dầu. Srm rửa sạch sâu quá khô 2 bênh c...   
...              ...                                                ...   
20617            218  Tạm, bôi lên bết rích khó chịu cực kì, phân kh...   
20631           1081  Lần đầu tiên xài nước tẩy trang , kem chống nắ...   
20634           1310  Kem rất bết rích, cảm giác nặng mặt, tán khó, ...   
20643           4145  Không có gì đáng phàn nàn, sản phẩm phù hợp vớ...   
20653           1818  Sản phẩm dùng ổn, không có gì để chê nhiều, đủ...   

      ngay_binh_luan gio_binh_luan  so_sao  ma_san_pham  \
36          9/6/2023         13:34       3    248700035   
53        15/07/2021          8:34       3    343700002   
69        15/04/2023         16:18       3    318900002   
86         5/10/2021         12:11       3    422200305   
104       20/08/2022         11:48       3    204900159   
...              ...           ...     ...          ...   
20617     29/03/2022         16:11       3    422216990   
20631       1/7/2022         19:49       3    422204904   
20634     25/03/2022         11:52       3    100160021   
20643       9/2/2022          8:49       3    422205163   
20653     17/04/2024         16:09       3    422216098   

                                             content_new  sentiment  
36                                      thắm hơi rửa xài          1  
53     mụn cân_nhắc chăm_sóc mụn bôi lâu_thấm bôi mỏn...          1  
69                               ẩm_ổn thâm mờ tốt cũ mờ          1  
86                                       mụn xài mát nhẹ          1  
104               rửa sạch sâu khô_bênh cánh nguyên căng          1  
...                                                  ...        ...  
20617                                  tạm bôi cực_kì rẻ          1  
20631                     xài tẩy chống hải_cái kích_ứng          1  
20634                        bết_rích mặt_tán đặc mặc_dù          1  
20643                                  phàn_nàn sản_phẩm          1  
20653                                             ổn chê          1  

[979 rows x 8 columns]

In [103]:
df1['sentiment'].value_counts()

sentiment
0    18650
1     2024
Name: count, dtype: int64

# Nhận xét:Dữ liệu có khả năng mất cân bằng  
Đánh giá tích cực là: 18650  
Đánh giá không tốt là: 2024

In [104]:
# Tách dữ liệu thành đầu vào (X) và nhãn (y)
X = df1['content_new']
y = df1['sentiment']

In [105]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [106]:
# Vector hóa văn bản bằng TF-IDF
vectorizer = TfidfVectorizer(max_features=5000) 
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [107]:
# Hàm huấn luyện và đánh giá mô hình
def train_and_evaluate_model(model, model_name):
    # Huấn luyện mô hình
    model.fit(X_train_tfidf, y_train)
    # Dự đoán
    y_pred = model.predict(X_test_tfidf)
    # Đánh giá mô hình
    print(f"=== {model_name} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [108]:
# 1. Logistic Regression
logistic_model = LogisticRegression(max_iter=1000)
train_and_evaluate_model(logistic_model, "Logistic Regression")

# 2. Support Vector Machine (SVM)
svm_model = SVC(kernel='linear', probability=True)
train_and_evaluate_model(svm_model, "Support Vector Machine")

# 3. Naive Bayes
naive_bayes_model = MultinomialNB()
train_and_evaluate_model(naive_bayes_model, "Naive Bayes")

=== Logistic Regression ===
Accuracy: 0.9409915356711004
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3730
           1       0.93      0.43      0.59       405

    accuracy                           0.94      4135
   macro avg       0.93      0.71      0.78      4135
weighted avg       0.94      0.94      0.93      4135

=== Support Vector Machine ===
Accuracy: 0.9535671100362757
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      3730
           1       0.90      0.59      0.71       405

    accuracy                           0.95      4135
   macro avg       0.93      0.79      0.84      4135
weighted avg       0.95      0.95      0.95      4135

=== Naive Bayes ===
Accuracy: 0.9363966142684401
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      3730
           1       0.98      0.36      0.52       405

    accuracy    